In [1]:
import numpy as np
import pandas as pd 
import csv 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import warnings
warnings.simplefilter("ignore")

In [2]:
#  loading the data set 
file = pd.read_csv('BSOM_DataSet_for_HW3.csv')


# to check whether there are any empty values or not 
df=file[['all_mcqs_avg_n20', 'all_NBME_avg_n4', 'CBSE_01',  'CBSE_02', 'LEVEL']]


df['LEVEL'].fillna(value='A', inplace=True) 

df[['CBSE_02']].fillna(method ='bfill')
#.fillna(52, inplace=True)
df.fillna(df['CBSE_02'].mean(), inplace=True)

df.isnull().any()


all_mcqs_avg_n20    False
all_NBME_avg_n4     False
CBSE_01             False
CBSE_02             False
LEVEL               False
dtype: bool

In [3]:
LEVEL = {'A': 0,'B': 1,'C':2,'D':3}

df.LEVEL = [LEVEL[item] for item in df.LEVEL] 
print(df) 

     all_mcqs_avg_n20  all_NBME_avg_n4  CBSE_01  CBSE_02  LEVEL
0               0.736           0.7700       42     68.0      1
1               0.740           0.8000       44     67.0      2
2               0.807           0.8125       41     78.0      0
3               0.886           0.9250       68     91.0      0
4               0.839           0.8550       57     74.0      1
..                ...              ...      ...      ...    ...
110             0.637           0.6825       45     52.0      2
111             0.751           0.8150       53     60.0      2
112             0.811           0.8325       50     77.0      1
113             0.729           0.7275       43     68.0      2
114             0.839           0.8625       54     73.0      1

[115 rows x 5 columns]


In [4]:
# Transforming y to binary number using one-hot- encoding


from sklearn.compose import ColumnTransformer

#onhtencder = OneHotEncoder(categorical_features = [0]) 

onhtencder= ColumnTransformer([("LEVEL", OneHotEncoder(), [0])], remainder = 'passthrough')

data = onhtencder.fit_transform(df[['LEVEL']]).toarray() 
#data

In [5]:

X= df.iloc[:,:]

X_norm = (X.iloc[:,:-1] - X.iloc[:,:-1].mean()) / (X.iloc[:,:-1].max() - X.iloc[:,:-1].min())



X_modified = pd.concat([X_norm, X.iloc[:,-1]],axis=1)

target=df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X_modified, data, test_size=0.20,random_state=9)

In [6]:

Train_ip=np.array(X_train.iloc[:,:-1])
Train_otpts=np.array(y_train)


In [7]:
# Initializing weights randomly
np.random.seed(45)


wt1 = np.random.random((1,5)) 


wt2 = np.random.random((2,2))  

wt3 = np.random.random((4,3))  


ttl_weits = np.hstack((wt1.ravel(order='F'), wt2.ravel(order='F'),wt3.ravel(order='F')))    

# layers configuration 

ip_lyr = 4
hd_lyr1 = 1
hd_lyr2 = 2
no_labls = 4

print('wt1',wt1)
print('wt2',wt2)
print('wt3',wt3)



wt1 [[0.98901151 0.54954473 0.2814473  0.07728957 0.4444695 ]]
wt2 [[0.47280797 0.048522  ]
 [0.16332445 0.11595071]]
wt3 [[0.62739168 0.85618205 0.65010242]
 [0.99072168 0.47035075 0.61829448]
 [0.28266721 0.97600332 0.673068  ]
 [0.44053089 0.28968734 0.50969968]]


In [8]:
def Weights(ttl_weits, ip_lyr, hd_lyr1,hd_lyr2, no_labls):

    
    w1 = np.reshape(ttl_weits[:hd_lyr1*(ip_lyr+1)], (hd_lyr1, ip_lyr+1), 'F')
    w2 = np.reshape(ttl_weits[hd_lyr1*(ip_lyr+1):(hd_lyr1*(ip_lyr+1)+hd_lyr2*(hd_lyr1+1))], (hd_lyr2, hd_lyr1+1), 'F')
    w3 = np.reshape(ttl_weits[(hd_lyr1*(ip_lyr+1)+hd_lyr2*(hd_lyr1+1)):], (no_labls, hd_lyr2+1), 'F')
 
   
    return w1,w2,w3

    

In [9]:


def SIG(z):
    return 1/(1+np.exp(-z))



def SIGGrad(z):
    return np.multiply(SIG(z), 1-SIG(z))

In [10]:
def Cost(h,y):
    cost = np.sum(np.multiply(y, np.log(h))+np.multiply(1-y, np.log(1-h)))
    return cost

In [11]:
def Feedfrwd(ttl_weits, ip_lyr, hd_lyr1,hd_lyr2, no_labls, X, y):
    
    wt1,wt2,wt3 = Weights(ttl_weits, ip_lyr, hd_lyr1,hd_lyr2, no_labls)
    
    l = len(Train_otpts)
   
    ones = np.ones((l,1))
    
    a1 = np.hstack((ones, Train_ip))
    
    a2 = SIG(a1 @ wt1.T)
    
    a2 = np.hstack((ones, a2))
    
    a3 = SIG(a2 @ wt2.T)
    
    a3 = np.hstack((ones, a3))
    
    h = SIG(a3 @ wt3.T)
    
    c=Cost(h,Train_otpts)
    
    
    return c / (-l)     
    

In [12]:
cf_er = Feedfrwd(ttl_weits, ip_lyr, hd_lyr1,hd_lyr2, no_labls, Train_ip,Train_otpts)


In [13]:
def NN(ttl_weits, ip_lyr, hd_lyr1,hd_lyr2, no_labls, X, y):
    

    initt_wt1,initt_wt2,initt_wt3 = Weights(ttl_weits, ip_lyr, hd_lyr1,hd_lyr2, no_labls)
    
   
    
    l = len(y)

    ones = np.ones((l,1))
    a1 = np.hstack((ones, X))
    #print('a1',a1)
    #print('a1',a1.shape)
    z2 = a1 @ initt_wt1.T
    #print('z2',z2)
    #print('z2',z2.shape)
    a2 = np.hstack((ones, SIG(z2)))
    #print('a2',a2)
    #print('a2',a2.shape)
    z3 = a2 @ initt_wt2.T
    #print('z3',z3)
    #print('z3',z3.shape)
    a3 = np.hstack((ones, SIG(z3)))
    #print('a3',a3)
    #print('a3',a3.shape)
    #print('',.shape)print('',)
    z4 = a3 @ initt_wt3.T
    a4 = SIG(z4)
    #print(a4.shape)
    d4 = a4-y

    #print('d4',d4.shape)
    #print('d4',d4)
    l = len(y)

    ones = np.ones((l,1))
    #print('z3',z3.shape)
    z3 = np.hstack((ones, z3))  

    #print('z3',z3.shape)
    #print('z3',z3)

    d3 = np.multiply(initt_wt3.T @ d4.T, SIGGrad(z3).T)
    #print('d3',d3.shape)
    #print('d3',d3)

    z2 = np.hstack((ones, z2))
    #print('z2',z2.shape)
    #print('z2',z2)
    d2 = np.multiply(initt_wt2.T @ d3[1:], SIGGrad(z2).T)
    #print('d2',d2.shape)
    #print('d2',d2)
    
    Delta1 = np.zeros(initt_wt1.shape)
    Delta2 = np.zeros(initt_wt2.shape)
    Delta3 = np.zeros(initt_wt3.shape)


    Delta1 = Delta1 + d2[1:,:] @ a1
    #print('Delta1',Delta1.shape)
    Delta2 = Delta2 + d3[1:,:] @ a2
    #print('Delta2',Delta2.shape)
    Delta3 = Delta3 + d4.T @ a3
    #print('Delta3',Delta3.shape)



    Delta1 /= l
    Delta2 /= l
    Delta3 /= l

        
    return np.hstack((Delta1.ravel(order='F'), Delta2.ravel(order='F'),Delta3.ravel(order='F'))) 

In [14]:
bckprop_wts = NN(ttl_weits, ip_lyr, hd_lyr1,hd_lyr2, no_labls,Train_ip,Train_otpts)



In [15]:
def predicting_y(wt1, wt2,wt3, X, y):
    l = len(y)
    ones = np.ones((l,1))
    a1 = np.hstack((ones, X))
    
    a2 = SIG(a1 @ wt1.T)
    
    a2 = np.hstack((ones, a2))
    
    a3 = SIG(a2 @ wt2.T)
    
    a3 = np.hstack((ones, a3))
    
    h = SIG(a3 @ wt3.T)
    
    
    
    return np.argmax(h, axis = 1)

In [16]:

def Gard_checking(ttl_weits,bckprop_wts,ip_lyr, hd_lyr1,hd_lyr2, no_labls,X,y):
    eps = 0.22
    
    
    
    
    l = len(ttl_weits) 
    

    for i in range(5):
        iterattion = int(np.random.rand()*len(ttl_weits))
        
        adding_eps = np.zeros((len(ttl_weits),1))
        
        adding_eps[iterattion] = eps
        

        ThetaPlus = Feedfrwd(ttl_weits + adding_eps.flatten(),ip_lyr, hd_lyr1,hd_lyr2, no_labls,X,y)
        
        #print('ThetaPlus',ThetaPlus)
        
        ThetaMinus  = Feedfrwd(ttl_weits - adding_eps.flatten(),ip_lyr, hd_lyr1,hd_lyr2, no_labls,X,y)
        #print('ThetaMinus',ThetaMinus)
        
        num_app = (ThetaPlus - ThetaMinus) / float(2*eps)
        print('num_app',num_app)
       
        print('bckprop_wts[iterattion]',bckprop_wts[iterattion])
        
        
        


In [17]:
grad_chk=Gard_checking(ttl_weits,bckprop_wts,ip_lyr, hd_lyr1,hd_lyr2, no_labls,Train_ip,Train_otpts)


grad_chk

num_app -0.00032673779177614585
bckprop_wts[iterattion] -0.00032708955654712037
num_app -0.0003183235363261745
bckprop_wts[iterattion] -0.0003185128053965527
num_app 0.4551996544921666
bckprop_wts[iterattion] 0.45594117199509143
num_app 0.31528402568731545
bckprop_wts[iterattion] 0.31644088486887084
num_app 0.23103084216977507
bckprop_wts[iterattion] 0.23152264063668462


In [18]:
import scipy.optimize as opt
Final_weights = opt.fmin_cg(maxiter = 100, f = Feedfrwd, x0 = ttl_weits, fprime = NN,
                        args = (ip_lyr, hd_lyr1,hd_lyr2, no_labls,Train_ip,Train_otpts))






         Current function value: 1.226025
         Iterations: 100
         Function evaluations: 211
         Gradient evaluations: 211


In [19]:
Final_wts1,Final_wts2,Final_wts3 = Weights(Final_weights,ip_lyr, hd_lyr1,hd_lyr2, no_labls)

## Predicting y values for test_data

In [20]:
pred = predicting_y(Final_wts1,Final_wts2,Final_wts3, X_test.iloc[:,:-1], y_test)
pred


array([2, 0, 0, 1, 1, 1, 2, 1, 1, 2, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0], dtype=int64)

In [21]:
y_actual=X_test.iloc[:,-1]


In [22]:


from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_actual,pred))

#print(accuracy_score(y_actual,pred))

from sklearn.metrics import accuracy_score
accuracy_score(y_actual,pred)
#print(type(y_test))

print(accuracy_score(y_actual,pred))


[[3 1 1 0]
 [4 7 0 0]
 [0 4 2 0]
 [0 1 0 0]]
0.5217391304347826


In [23]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


print('f1_score',f1_score(y_actual,pred, average="weighted"))
print('precision_score',precision_score(y_actual,pred, average="weighted"))
print('recall_score',recall_score(y_actual,pred, average="weighted"))  

from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y_actual,pred, pos_label=2)
print('AUC score :',metrics.auc(fpr, tpr))

f1_score 0.5036231884057971
precision_score 0.524605828953655
recall_score 0.5217391304347826
AUC score : 0.7745098039215685
